In [30]:
import pandas as pd
import torch
import random
import torch.nn.functional as F



In [4]:
names = open('names.csv').readlines()
words = []

In [5]:
for name in names:
    words.append(name.split(','))
for i in range(len(words)):
    words[i] = words[i][1].lower()
words.pop(0)


'name'

In [6]:
chars = sorted(list(set(''.join(words))))
stoi =  {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [9]:

block_size = 4
def build_dataset(words):
    x,y = [],[]
    for w in words:
        context = [0] * block_size
        for ch in w+ '.':
            ix = stoi[ch]
            x.append(context)
            y.append(ix)
            context = context[1:] + [ix]

    x = torch.tensor(x)
    y = torch.tensor(y)
    return x,y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

xtr,ytr = build_dataset(words[:n1])
xdev, ydev = build_dataset(words[n1:n2])
xte, yte = build_dataset(words[n2:])


In [26]:
g = torch.Generator().manual_seed(42)
c = torch.randn((27,10), generator=g)
w1 = torch.randn((40,300),generator=g)
b1 = torch.randn((300),generator=g)
w2 = torch.randn((300,27),generator=g)
b2 = torch.randn((27),generator=g)
parameters = [c,w1,b1,w2,b2]


In [27]:
sum(p.nelement() for p in parameters)

20697

In [28]:
for p in parameters:
    p.requires_grad = True

In [29]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre
lri = []
lossi = []
stepi = []

In [41]:
for i in range(20000):
    ix = torch.randint(0, xtr.shape[0], (100,))

    emb = c[xtr[ix]]
    h = torch.tanh(emb.view(-1,40) @ w1 +b1)
    logits = h @ w2 +b2
    loss = F.cross_entropy(logits,ytr[ix])
    print(loss.item())

    for p in parameters:
        p.grad = None
    loss.backward()

    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    

2.0523884296417236
2.0246362686157227
2.2427966594696045
2.177769422531128
2.0056841373443604
2.0078024864196777
2.246671438217163
2.130793333053589
2.108059883117676
2.3417434692382812
2.2655255794525146
2.4134976863861084
2.166355848312378
2.3078501224517822
2.2408089637756348
2.3065292835235596
2.166937828063965
2.1452345848083496
2.2450830936431885
2.152815818786621
2.043731451034546
2.0424718856811523
2.049463987350464
2.1639761924743652
1.968287467956543
2.455868721008301
2.319147825241089
2.275468587875366
2.0703351497650146
1.9787511825561523
2.265082836151123
2.108548164367676
2.059009552001953
2.216062545776367
2.4222021102905273
2.28670334815979
2.3204119205474854
2.2183542251586914
2.1710283756256104
2.198392868041992
2.0267724990844727
2.1510190963745117
2.5584964752197266
2.3782355785369873
2.228236198425293
2.224614143371582
2.1899895668029785
2.404965400695801
1.8840880393981934
1.9465855360031128
1.9980720281600952
2.284895896911621
2.106515645980835
2.3352959156036377

In [40]:

for _ in range(20):
    out = []
    context = [0] * block_size 
    while True:
      emb = c[torch.tensor([context])]
      h = torch.tanh(emb.view(1, -1) @ w1 + b1)
      logits = h @ w2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

kamarese.
amman.
satar.
avannetterijeen.
mavone.
emine.
kaora.
rashed.
avenie.
hest.
pelet.
makan.
mang.
tacie.
alannite.
noke.
shaterin.
anco.
asile.
noee.
